In [ ]:
# Aim:
## Todo: 
# multiprocessing

## Convertor

In [ ]:
# Vivek Kumar
# Initial ground work
# https://medium.com/@vivek8981/dicom-to-jpg-and-extract-all-patients-information-using-python-5e6dd1f1a07d
# but the images loss their contrast

# Pillow lacks the JPEG 2000 plugin
# this need JPEG 2000 support
# pixel_array = ds.pixel_array.astype(float)  # preparing for scaling
# pip install pylibjpeg-libjpeg
# pip install pylibjpeg pylibjpeg-libjpeg pydicom

# VOI LUT problem
# Window level problem
#  most of DICOM's store pixel values in exponential scale, which is resolved by standard standard DICOM viewers.
# best solution for xray
###https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
# Good explanation on VOI LUT
### https://www.leadtools.com/help/sdk/v21/dh/to/working-with-dicom-lut.html
# pydicom.pixel_data_handlers.util import apply_voi_lut
# apply_voi_lut(array, ds, prefer_lut=True)

# CT file use window/level, insead of VOI LUT to modify final image
# window/level may express in a list (pydicom.multival.MultiValue)
# Xray usually have window/level
# if no, have to add LUT transfer function
# if MONOCHROME1 -> have to inverse

# CR 如果沒有 window/center 必須要有 VOI sequence
# CR 有 window/center 則通常沒有　voi
# CT window/center 可以有　pydicom.multival.MultiValue or pydicom.valuerep.DSfloat
# CT 沒在用 VOI

In [407]:
import pydicom
import cv2
import pandas
from PIL import Image
import numpy as np
from pathlib import Path
import os
from pydicom.pixel_data_handlers.util import apply_voi_lut
import time

In [2]:
# Get dicom files into a list

folder_origin = Path.home() / 'Desktop' / 'dicom_test'
folder_destination = Path.home() / 'Desktop' / 'dicom_test_jpg'

folder_origin = Path("E:\Temp\General")
folder_destination = Path("E:\dicom_test_jpg")

dicom_file_list = []
# export_file_list = []

# read file
for root, sub_f, file in os.walk(folder_origin):
    for f in file:
        if f.endswith('dcm'):
            file_path_dcm = Path(root)/Path(f)
            # file_path_exp =  folder_destination / Path(f).with_suffix('.jpg')
            # stor origin / destination
            dicom_file_list.append(file_path_dcm)
# sort the list
dicom_file_list.sort()


## Functionalize

In [443]:
dicom2jpg('E:\\Temp\\General\\4673229-0',target_root='E:\\', filetype='jpg')

error: OpenCV(4.0.1) C:\ci\opencv-suite_1573470242804\work\modules\imgcodecs\src\loadsave.cpp:667: error: (-215:Assertion failed) image.channels() == 1 || image.channels() == 3 || image.channels() == 4 in function 'cv::imwrite_'


In [436]:

def get_root_get_dicom_file_list(origin):
    # if single file, return root folder of origin file and a list of that file
    origin = Path(origin)
    dicom_file_list = []
    
    # if file or folder does not exist
    if not origin.exists():
        raise OSError('File or folder does not exist')
    # if it is a file
    if origin.is_file():
        if origin.suffix.lower()!='.dcm':
            raise Exception('Input file type should be a DICOM file')
        else:
            dicom_file_list = [origin]
            root_folder = origin.parent
    # if it is a folder
    else:
        # read file
        for root, sub_f, file in os.walk(origin):
            for f in file:
                if f.lower().endswith('.dcm'):
                    file_path_dcm = Path(root)/Path(f)
                    # file_path_exp =  folder_destination / Path(f).with_suffix('.jpg')
                    # stor origin / destination
                    dicom_file_list.append(file_path_dcm)
        # sort the list
        dicom_file_list.sort()
        # foot folder
        root_folder = origin.parent
    return root_folder, dicom_file_list


def dicom2jpg(origin, target_root=None, filetype=None):
    """
    origin: can be a .dcm file or a folder
    target_root: root of output files and folders; default: root of origin file or folder
    filetype: can be jpg, jpeg, tiff, png, or bmp
    full target file path = target_root/Today/PatientID_filetype/StudyDate_StudyTime_Modality_AccNum/Ser_Img.filetype
    """
    # set file type
    if filetype is None:
        filetype = 'jpg'
    elif filetype.lower() not in ['jpg','tiff','png', 'jpeg', 'bmp']:
        raise Exception('Target file type should be jpg, tiff, png, or bmp')
    
    # get root folder and dicom_file_list
    root_folder, dicom_file_list = get_root_get_dicom_file_list(origin)

    # if target root is not specified, set as same root of origin file 
    if target_root is None:
        target_root = root_folder
    else:
        target_root = Path(target_root)

    # Iterate through all dicom files
    for file_path in dicom_file_list:
        # read images and their pixel data
        ds = pydicom.dcmread(file_path, force=True)
        # convert pixel_array (img) to -> gray image 
        # Need JPEG 2000 plugin for pillow
        pixel_array = ds.pixel_array.astype(float)  # preparing for scaling

        # rescale slope, rescale intercept, adjust window and level
        try:
            rescale_slope = int(ds.RescaleSlope)
            rescale_intercept = int(ds.RescaleIntercept)
            pixel_array = (pixel_array)*rescale_slope+rescale_intercept
        except:
            pass

        # Adjust according to LUT, window center(level) and width values
        # xxx=np.piecewise(x, [condition1,condition2], [func1,func2])
        def get_LUT_value(data, window, level):
            return np.piecewise(data, 
                [data<=(level-0.5-(window-1)/2),
                data>(level-0.5+(window-1)/2)],
                [0,255,lambda data: ((data-(level-0.5))/(window-1)+0.5)*(255-0)])

        # get window center and window width value
        try:
            window_center = ds.WindowCenter
            window_width = ds.WindowWidth
            # some values may be stored in an array
            if type(window_center)==pydicom.multival.MultiValue:
                window_center = int(window_center[0])
            else:
                window_center = int(window_center)
            if type(window_width)==pydicom.multival.MultiValue:
                window_width = int(window_width[0])
            else:
                window_width = int(window_width)
            pixel_array = get_LUT_value(pixel_array, window_width, window_center)
        except:
        # if there is no window center, window width tag, try obtaining VOI LUT setting (usually happens to plain films)
            try:
                if ds.VOILUTSequence:
                    pixel_array = apply_voi_lut(ds.pixel_array, ds)
            except:
                pass

        # normalize to 8bit information
        # Conver to uint8 (8-bit unsigned integer), for image to save/display
        # almost no difference. However, this formula yeild slightly more standard deviation 
        pixel_array = ((pixel_array-pixel_array.min())/(pixel_array.max()-pixel_array.min())) * 255.0

        # These 2 formula are the same
        # pixel_array = (np.maximum(pixel_array,0) / pixel_array.max()) * 255.0
        #pixel_array = pixel_array - np.min(pixel_array)
        #pixel_array = pixel_array / np.max(pixel_array)
        #pixel_array = (pixel_array*255).astype(np.uint8)

        # if PhotometricInterpretation == "MONOCHROME1", then inverse; eg. xrays
        try:
            if ds.PhotometricInterpretation == "MONOCHROME1":
            # if ds.PresentationLUTShape=='INVERSE':  # not always be shown
            # NOT add minus directly
                pixel_array = np.max(pixel_array) - pixel_array
        except:
            pass
        
        # Try to get metadata        
        try:
            StudyDate = ds.StudyDate  # study date
        except:
            StudyDate = 'UnknownDate'
        try:
            StudyTime = ds.StudyTime.split('.')[0] # study time
        except:
            StudyTime = 'UnknownTime'
        try:
            AccessionNumber = ds.AccessionNumber  # Acc number
        except:
            AccessionNumber = 'UnknownAccNum'
        try:
            Modality = ds.Modality  # modality
        except:
            Modality = 'UnknownModality'
        try:
            PatientID = ds.PatientID  # patient id
        except:
            PatientID = 'UnknownID'
        try:
            SeriesNumber = ds.SeriesNumber  # series number
        except:
            SeriesNumber = 'Ser'
        try:
            InstanceNumber = ds.InstanceNumber
        except:
            InstanceNumber = 'Ins'
            
        # Full export file path
        # target_root/Today/PatientID_filetype/StudyDate_StudyTime_Modality_AccNum/Ser_Img.filetype
        today_str = time.strftime('%Y%m%d')
        full_export_fp_fn = target_root/Path(today_str)/Path(f"{PatientID}_{filetype}")/Path(f"{StudyDate}_{StudyTime}_{Modality}_{AccessionNumber}")/Path(f"{SeriesNumber}_{InstanceNumber}.{filetype}")
        # make dir
        Path.mkdir(full_export_fp_fn.parent, exist_ok=True, parents=True)
        # write file
        cv2.imwrite(str(full_export_fp_fn), pixel_array)




## Yet to functionalize

In [216]:
for file_path in dicom_file_list[:20]:
    # read images and their pixel data
    ds = pydicom.dcmread(file_path, force=True)
    # convert pixel_array (img) to -> gray image 
    # Need JPEG 2000 plugin for pillow
    pixel_array = ds.pixel_array.astype(float)  # preparing for scaling
    
    # rescale slope, rescale intercept, adjust window and level
    try:
        rescale_slope = int(ds.RescaleSlope)
        rescale_intercept = int(ds.RescaleIntercept)
        pixel_array = (pixel_array)*rescale_slope+rescale_intercept
    except:
        pass

    # Adjust according to LUT, window center(level) and width values
    # xxx=np.piecewise(x, [condition1,condition2], [func1,func2])
    def get_LUT_value(data, window, level):
        return np.piecewise(data, 
            [data<=(level-0.5-(window-1)/2),
            data>(level-0.5+(window-1)/2)],
            [0,255,lambda data: ((data-(level-0.5))/(window-1)+0.5)*(255-0)])
    
    # get window center and window width value
    try:
        window_center = ds.WindowCenter
        window_width = ds.WindowWidth
        # some values may be stored in an array
        if type(window_center)==pydicom.multival.MultiValue:
            window_center = int(window_center[0])
        else:
            window_center = int(window_center)
        if type(window_width)==pydicom.multival.MultiValue:
            window_width = int(window_width[0])
        else:
            window_width = int(window_width)
        pixel_array = get_LUT_value(pixel_array, window_width, window_center)
    except:
    # if there is no window center, window width tag, try obtaining VOI LUT setting (usually happens to plain films)
        try:
            if ds.VOILUTSequence:
                pixel_array = apply_voi_lut(ds.pixel_array, ds)
        except:
            pass
        
    # normalize to 8bit information
    # Conver to uint8 (8-bit unsigned integer), for image to save/display
    # almost no difference
    # SD: 77.15839533934914
    pixel_array = ((pixel_array-pixel_array.min())/(pixel_array.max()-pixel_array.min())) * 255.0
    
    # SD: 76.70618496373392
    # pixel_array = (np.maximum(pixel_array,0) / pixel_array.max()) * 255.0
    # SD: 77.15839533934914
    #pixel_array = pixel_array - np.min(pixel_array)
    #pixel_array = pixel_array / np.max(pixel_array)
    #pixel_array = (pixel_array*255).astype(np.uint8)
    
    # if PhotometricInterpretation == "MONOCHROME1", then inverse; eg. xrays
    try:
        if ds.PhotometricInterpretation == "MONOCHROME1":
        # if ds.PresentationLUTShape=='INVERSE':  # not always be shown
        # NOT add minus directly
            pixel_array = np.max(pixel_array) - pixel_array
    except:
        pass

    # export filepath/filename = folder_destination / full_file_name.dcm -> jpg/png/tiff/bmp
    full_export_fp_fn = folder_destination / file_path.with_suffix('.JPG').name
    # full_export_fp_fn = folder_destination / file_path.with_suffix('.png').name    
    # full_export_fp_fn = folder_destination / file_path.with_suffix('.tiff').name
    
    # make dir
    Path.mkdir(full_export_fp_fn.parent, exist_ok=True)
    # write file
    cv2.imwrite(str(full_export_fp_fn), pixel_array)
    



### Previous version

In [ ]:
# to figure out the pattern and relationship between VOI, LUT, window level/width
# some has VOI LUT, some has not
# some 
for file_path in dicom_file_list:
    ds = pydicom.dcmread(file_path, force=True)
    
    try:
        window_w = ds.WindowWidth
    except:
        window_w = 'x'
    try:
        window_c = ds.WindowCenter
    except:
        window_c = 'x'
    
    try:
        if ds.VOILUTSequence:
            voi_seq = '0'
        else:
            voi_seq = 'N'
    except:
        voi_seq='x'
        
    if ds.Modality=='CT' and type(ds.WindowWidth)==pydicom.multival.MultiValue:
        print(f'{ds.Modality} - window_w={window_w}, window_c={window_c}, {type(window_w)}, voi={voi_seq}')
        print(int(window_w[0]))
    
    # CR 如果沒有 window/center 必須要有 VOI sequence
    # CR 有 window/center 則通常沒有　voi
    # CT window/center 可以有　pydicom.multival.MultiValue or pydicom.valuerep.DSfloat
    # CT 沒在用 VOI

In [ ]:
# to solve VOI, LUT problem
# method in 
# https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way

# Get VOI LUT setting
try:
    if ds.VOILUTSequence:
        voi_lut = True
    else:
        voi_lut = False
except:
    voi_lut = False

# VOI LUT (if available by DICOM device) is used to transform raw DICOM data to "human-friendly" view
if voi_lut:
    pixel_array = apply_voi_lut(ds.pixel_array, ds)
else:
    pixel_array = ds.pixel_array

# if PhotometricInterpretation == "MONOCHROME1", then inverse
try:
    if ds.PhotometricInterpretation == "MONOCHROME1":
    # if ds.PresentationLUTShape=='INVERSE':  # not necessary written
        pixel_array = np.amax(pixel_array) - pixel_array
except:
    pass
    
# scale to 255
pixel_array = pixel_array - np.min(pixel_array)
pixel_array = pixel_array / np.max(pixel_array)
pixel_array = (pixel_array * 255).astype(np.uint8)

display(Image.fromarray(np.uint8(pixel_array)))

In [ ]:
# Auto-convertor test function
# Need to solve the window/level/slope

dicom_file_list = dicom_file_list[:100]
# dicom_file_list = [Path('E:/Temp/General/000103436C/1.2.124.113532.10.21.11.4.20210601.113528.240164/1.3.51.0.7.13927215276.48758.24141.44523.34501.26226.49959/1.3.51.0.7.11650386957.16987.45636.41431.33978.40699.39256.dcm')]

for file_path in dicom_file_list:
    # read images and their pixel data
    ds = pydicom.dcmread(file_path, force=True)
    # # Convert pixel_array (img) to -> gray image (img_2d_scaled)
    # Pillow lacks the JPEG 2000 plugin
    # pip install pylibjpeg-libjpeg
    # pip install pylibjpeg pylibjpeg-libjpeg pydicom
    pixel_array = ds.pixel_array.astype(float)  # preparing for scaling

    
    # rescale slope, rescale intercept, adjust window and level
    try:
        rescale_slope = int(ds.RescaleSlope)
        rescale_intercept = int(ds.RescaleIntercept)
        pixel_array = (pixel_array)*rescale_slope+rescale_intercept
    except:
        pass

    # Adjust according to LUT
    # xxx=np.piecewise(x, [condition1,condition2], [func1,func2])
    # already 
    def get_LUT_value(data, window, level):
        return np.piecewise(data, 
            [data <= (level - 0.5 - (window-1)/2),
                data > (level - 0.5 + (window-1)/2)],
                [0, 255, lambda data: ((data - (level - 0.5))/(window-1) + 0.5)*(255-0)])
    
    # get window center and window width value
    try:
        window_center = int(ds.WindowCenter)
        window_width = int(ds.WindowWidth)
        pixel_array = get_LUT_value(pixel_array, window_width, window_center)
    except:
        pass
    
    
    
    # if inverse
    try:
        # inverse_sign = -1 if ds.PresentationLUTShape=='INVERSE' else 1
        inverse_sign = -1 if ds.PhotometricInterpretation == "MONOCHROME1" else 1
        pixel_array = inverse_sign * pixel_array  # *-1 is not good!! 
        # pixel_array = np.amax(pixel_array) - pixel_array is better 
    except:
        pass
    
    # Conver to uint8 (8-bit unsigned integer), for image to save/display
    # pixel_array_scaled = np.uint8(pixel_array)
    # Already done?
    
    # pixel_array_scaled = cv2.convertScaleAbs(pixel_array-np.min(pixel_array), alpha=(255.0 / min(np.max(pixel_array)-np.min(pixel_array), 10000)))
    # pixel_array_scaled = ((pixel_array-pixel_array.min())/(pixel_array.max()-pixel_array.min())) * 255.0
    
    # used in XRAY
    # pixel_array_scaled = (np.maximum(pixel_array, 0) / pixel_array.max()) * 255
    
    
    # export filepath/filename = folder_destination / full_file_name.dcm -> jpg
    full_export_fp_fn = folder_destination / file_path.with_suffix('.jpg').name

    # make dir
    Path.mkdir(full_export_fp_fn.parent, exist_ok=True)

    cv2.imwrite(str(full_export_fp_fn), pixel_array)
    


In [ ]:
# display CT file
# CT file use window/level, insead of VOI LUT to modify final image
# window/level may express in a list

ds = pydicom.dcmread(dicom_file_list[0], force=True)
pixel_array = ds.pixel_array.astype(float)  

# pixel_array = (pixel_array)*rescale_slope+rescale_intercept


# window center/level conversion function
# https://stackoverflow.com/questions/44924335/converting-12-bit-dicom-image-to-8-bit-jpeg
# http://dicom.nema.org/medical/dicom/current/output/chtml/part03/sect_C.11.2.html#sect_C.11.2.1.2
def get_LUT_value(data, window, level):
    return np.piecewise(data, 
        [data <= (level - 0.5 - (window-1)/2),
            data > (level - 0.5 + (window-1)/2)],
            [0, 255, lambda data: ((data - (level - 0.5))/(window-1) + 0.5)*(255-0)])
pixel_array = get_LUT_value(pixel_array, window_width, window_center)


pixel_array =  (np.maximum(pixel_array, 0) / pixel_array.max()) * 255
#pixel_array = cv2.convertScaleAbs(pixel_array-window_center, alpha=(255.0/window_width))

display(Image.fromarray(np.uint8(pixel_array)))

In [ ]:
# use and tested this in XRAY
# Xray usually have window/level
# if no, have to add LUT transfer function
# if MONOCHROME1 -> have to inverse

# from matplotlib import pyplot as plt
ds = pydicom.dcmread(dicom_file_list[0], force=True)
pixel_array = ds.pixel_array.astype(float)  


# pixel_array = cv2.convertScaleAbs(pixel_array-np.min(pixel_array), alpha=(255.0 / min(np.max(pixel_array)-np.min(pixel_array), 10000)))
# pixel_array = ((pixel_array-pixel_array.min())/(pixel_array.max()-pixel_array.min())) * 255.0
pixel_array =  (np.maximum(pixel_array, 0) / pixel_array.max()) * 255


# the use of Presentation LUT Shape == 'INVERSE' or PhotometricInterpretation == "MONOCHROME1"
# https://groups.google.com/g/comp.protocols.dicom/c/Wzt_06RbF5c?pli=1
# Presentation LUT Shape is mandatory in the DX Image Module
# Bottom line: both MONOCHROME2/IDENTITY and MONOCHROME1/INVERSE result in an output image where 0 is black and the maximum value is white (i.e. P-Values).
try:
    inverse_sign = -1 if ds.PresentationLUTShape=='INVERSE' else 1
    pixel_array = inverse_sign * pixel_array
except:
    pass
    
display(Image.fromarray(np.uint8(pixel_array)))

In [54]:
# But the contrast and 

dicom_file_list = dicom_file_list[:10]

for file_path in dicom_file_list:
    # read images and their pixel data
    ds = pydicom.dcmread(file_path)
    pixel_array = ds.pixel_array

    # export filepath/filename = folder_destination / full_file_name.dcm -> jpg
    full_export_fp_fn = folder_destination / file_path.with_suffix('.jpg').name

    # make dir
    Path.mkdir(full_export_fp_fn.parent, exist_ok=True)
    
    # export image
    cv2.imwrite(str(full_export_fp_fn), pixel_array)
    


In [2]:
from pydicom import dcmread
from pydicom.data import get_testdata_file

ds = dcmread(get_testdata_file('JPEG-LL.dcm'))
arr = ds.pixel_array

## Multiprocessing

In [1]:
import multiprocessing
cpus = multiprocessing.cpu_count()
cpus

8